In [1]:
# Colab 환경에서 필요한 패키지 설치
!pip install --quiet pandas numpy scikit-learn tqdm


In [7]:
# 1) Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2) 패키지 임포트 및 경로 설정
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Colab 상 드라이브 내 프로젝트 루트
base_dir = '/content/drive/MyDrive/Colab Notebooks/패턴인식'

# 입력(raw train) 및 출력 디렉터리/파일 경로
input_path  = os.path.join(base_dir, 'data', 'train.csv')
output_dir  = os.path.join(base_dir, 'data_preprocessing', 'result')
output_path = os.path.join(output_dir, 'trial1_train.csv')

# 출력 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# 3) 데이터 로드 및 타겟 생성
df = pd.read_csv(input_path)
df['target'] = (df['shares'] > 1400).astype(int)
df = df.drop(columns=['id', 'shares', 'y'], errors='ignore')

# 4) 수치형 및 범주형 컬럼 분리
categorical_cols = ['data_channel', 'weekday']
numerical_cols   = [col for col in df.columns if col not in categorical_cols + ['target']]

# 5) 전처리 파이프라인 정의
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler',  StandardScaler())
])
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline,   numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# 6) 피처 변환 및 DataFrame 복원
X   = df.drop(columns=['target'])
y   = df['target'].values
X_p = preprocessor.fit_transform(X)

# 원-핫 인코딩된 카테고리 특성명
feat_cat = preprocessor.named_transformers_['cat']['onehot']\
                      .get_feature_names_out(categorical_cols)
# 전체 특성명 결합
feature_names = numerical_cols + feat_cat.tolist()

# numpy array → DataFrame
df_processed = pd.DataFrame(X_p, columns=feature_names)
df_processed['target'] = y

# 7) 결과 저장
df_processed.to_csv(output_path, index=False)
print(f"✅ 생성 완료: {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 생성 완료: /content/drive/MyDrive/Colab Notebooks/패턴인식/data_preprocessing/result/trial1_train.csv
